In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
import re
import seaborn as sns
from sklearn.metrics import confusion_matrix
from zipfile import ZipFile
from glob import glob
from google.colab import drive
from keras.models import Sequential
from PIL import Image
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.layers import Dropout,BatchNormalization
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adagrad
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.metrics import auc


drive.mount('/content/drive')

file_name = '/content/drive/My Drive/Final Project DL & ML/chest-xray.zip'

with ZipFile(file_name, 'r') as zip_file:
  zip_file.extractall()  # Extract all contents of the ZIP file.
  print('Done')

train_path = '/content/chest_xray/chest_xray/train'
val_path = '/content/chest_xray/chest_xray/val'
test_path = '/content/chest_xray/chest_xray/test'

pre_path = '/content/chest_xray/chest_xray/'


# Define paths for different sets and classes.
train_normal_dir = pre_path + 'train/NORMAL/'
train_pneu_dir = pre_path + 'train/PNEUMONIA/'

test_normal_dir = pre_path + 'test/NORMAL/'
test_pneu_dir = pre_path + 'test/PNEUMONIA/'

val_normal_dir = pre_path + 'val/NORMAL/'
val_pneu_dir = pre_path + 'val/PNEUMONIA/'


#initiliazing the healthy, bacteria and virus cases of the images
virus = [] #1493 images total
bacteria = [] #2780 images total
healthy = [] #1583 images total

healthy += glob(train_normal_dir + '*jpeg')

#By using Regular Expressions we sorting the cases properly
#0- healthy, 1- virus case, 2 - bacteria case
for i in os.listdir(train_pneu_dir):
  if(re.search("virus.*jpeg", i)):
      virus.append([train_pneu_dir+i,1])
  elif(re.search("bacteria.*jpeg", i)):
      bacteria.append([train_pneu_dir+i,2])

healthy += glob(test_normal_dir + '*jpeg')

for i in os.listdir(test_pneu_dir):
  if(re.search("virus.*jpeg", i)):
      virus.append([test_pneu_dir+i,1])
  elif(re.search("bacteria.*jpeg", i)):
      bacteria.append([test_pneu_dir+i,2])

healthy += glob(val_normal_dir + '*jpeg')

for i in os.listdir(val_pneu_dir):
  if(re.search("virus.*jpeg", i)):
      virus.append([val_pneu_dir+i,1])
  elif(re.search("bacteria.*jpeg", i)):
      bacteria.append([val_pneu_dir+i,2])

for i in range(len(healthy)):
  healthy[i]=(healthy[i],0)

for lst in [healthy,virus,bacteria]:
  random.shuffle(lst)

# print(len(healthy))
# print(len(virus))
# print(len(bacteria))

# """# Creating the test, train and validation sets"""
# # 15% validation(878) and 20% test sets(1172), 65% train set from all data(3806)
# # Splitting data into train, validation, and test sets

val = virus[:250] + bacteria[:450] + healthy[:178]
test = virus[250:500] + bacteria[450:900] + healthy[178:650]
train = virus[500:] + bacteria[900:] + healthy[650:]


def NormalPixels_Labels(data):
    # normalizing the pixels i=on every image and label
    normal = []
    labels = []
    for im_path, label in data:
        if label == 2:
           #works only for Q1
            labels.append(1)
        else:
            labels.append(label)
        image = Image.open(im_path)  # Open image
        image = image.convert("L")   # Convert to grayscale
        resized_image = image.resize((180,180))  # Resize the image
        image_array = np.array(resized_image)  # Convert to NumPy array
        im_normal = image_array / 255.0  # Normalize pixel values
        normal.append(im_normal)
    return np.array(normal), np.array(labels)


def CreatingtVal(val):
    val_norm, val_label = NormalPixels_Labels(val)  # Normalize validation data
    return val_norm, val_label

def CreatingtTrain(train):
    train_norm, train_label = NormalPixels_Labels(train)  # Normalize training data
    return train_norm, train_label

def CreatingtTest(test):
    test_norm, test_label = NormalPixels_Labels(test)  # Normalize test data
    return test_norm, test_label


# Load and preprocess the data
val_norm, val_label = CreatingtVal(val)
train_norm, train_label = CreatingtTrain(train)
test_norm, test_label = CreatingtTest(test)






In [ ]:
# Build the model
test_accs = []
test_loss_accs = []
epochs = [10,15,20]
learning_rates = [0.001, 0.0001, 0.00001]


for lr in learning_rates:
  for epoch in epochs:
        print(f"Training CNN with SGD: learning rate: {lr}, epochs: {epoch}")
        CNN = models.Sequential()

        CNN.add(layers.Conv2D(32, (3, 3), padding = 'same' ,activation='relu', input_shape=(180, 180, 1)))
        CNN.add(BatchNormalization())
        CNN.add(layers.MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
        CNN.add(layers.Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))

        CNN.add(layers.MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
        CNN.add(layers.Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
        CNN.add(Dropout(0.1))
        CNN.add(BatchNormalization())

        CNN.add(layers.MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
        CNN.add(layers.Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
        CNN.add(BatchNormalization())

        CNN.add(layers.MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
        CNN.add(layers.Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
        CNN.add(Dropout(0.2))
        CNN.add(BatchNormalization())

        CNN.add(layers.MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
        CNN.add(layers.Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
        CNN.add(Dropout(0.3))
        CNN.add(BatchNormalization())

        CNN.add(layers.MaxPooling2D((2,2) , strides = 2 , padding = 'same'))

        CNN.add(layers.Flatten())
        CNN.add(Dropout(0.3))
        CNN.add(layers.Dense(1, activation = 'sigmoid'))

        # Compile the model
        CNN.compile(optimizer = optimizers.Adam(learning_rate=lr),loss='binary_crossentropy', metrics=['accuracy'])
        CNN.summary()
        history = CNN.fit(train_norm, train_label, epochs = epoch, batch_size = 20,
                            validation_data=(val_norm, val_label), verbose=1)
        # Evaluate the model on the test set
        test_loss, test_acc = CNN.evaluate(test_norm, test_label, verbose=1)
        test_accs.append((test_acc,f' Learning Rate: {lr}, Epochs: {epoch}'))
        test_loss_accs.append((test_loss,f'Learning Rate: {lr}, Epochs: {epoch}'))
        print("\nTest accuracy: ", test_acc)
        print("\nTest loss: ", test_loss)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'Training and Validation Accuracy vs. Number of Epochs\nOptimizer: Optimizer:SGD, Learning Rate: {lr}, Epochs: {epoch}')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.grid()
        plt.legend(loc='lower right')
        plt.show()

        plt.clf() # clear figure
        plt.plot(history.history['loss'], label='Training loss')
        # b is for "solid blue line"
        plt.plot(history.history['val_loss'], label='Validation loss')
        plt.title(f'Training and Validation Loss vs. Number of Epochs\nOptimizer:SGD, Learning Rate: {lr}, Epochs: {epoch}')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.grid()
        plt.legend(loc='upper right')
        plt.show()